<a href="https://colab.research.google.com/github/Hidestament/AtCoder/blob/main/ABC/ABC202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [AtCoder Beginner Contest 202](https://atcoder.jp/contests/abc202)

## [A - Three Dice](https://atcoder.jp/contests/abc202/tasks/abc202_a)

In [ ]:
a, b, c = map(int, input().split())
print(21 - (a + b + c))

## [B - 180°](https://atcoder.jp/contests/abc202/tasks/abc202_b)

In [ ]:
S = str(input())
S = S.replace("6", "a")
S = S.replace("9", "b")
S = S.replace("a", "9")
S = S.replace("b", "6")
print(S[::-1])

## [C - Made Up](https://atcoder.jp/contests/abc202/tasks/abc202_c)

In [ ]:
from collections import Counter

N = int(input())
A = list(map(int, input().split()))
B = list(map(int, input().split()))
C = list(map(int, input().split()))

In [ ]:
BC = []
for i in range(N):
    BC.append(B[C[i] - 1])

In [ ]:
cnt = Counter(BC)
ans = 0
for i in range(N):
    ans += cnt[A[i]]
print(ans)

## [D - aab aba baa](https://atcoder.jp/contests/abc202/tasks/abc202_d)

In [ ]:
aabb、abab、abba、baab、baba、bbaa
1桁目
a ~ の個数は 3C1 = 3
-> 1桁目はbに決定

K = 1
a, a, b
a ~ の個数は 2C1 = 2
-> 2桁目は a 決定


K = 1
a, b
a ~ の 個数は 1
-> 3桁目は a に決定

In [ ]:
A, B, K = map(int, input().split())

In [ ]:
from scipy.special import comb
ans = ""
def make_number(a, b, k):
    global ans
    if a == 0 and b == 0:
        return
    elif a == 0 or b == 0:
        ans += "a" * a
        ans += "b" * b
        return
    # 現在の桁をaにしたときの, それ以降の数の個数
    else:
        cnt = comb(a-1+b, a-1, exact=True)
        if k > cnt:
            ans += "b"
            return make_number(a, b-1, k-cnt)
        else:
            ans += "a"
            return make_number(a-1, b, k)

In [ ]:
make_number(A, B, K)
print(ans)

## [E - Count Descendants](https://atcoder.jp/contests/abc202/tasks/abc202_e)

### 解法1: オイラーツアー + 二分探索
PythonでAC

In [ ]:
import sys
sys.setrecursionlimit(10**7)
N = int(input())
P = list(map(int, input().split()))
Q = int(input())

7
1 1 2 2 4 2
4


In [ ]:
graph = [[] for _ in range(N)]
for i, p in enumerate(P, start=1):
    graph[p-1].append(i)

In [ ]:
# オイラーツアーとdepthの計算
euler_tour = []
depth = [0] * N
def dfs(now):
    euler_tour.append(now)
    for to in graph[now]:
        depth[to] = depth[now] + 1
        dfs(to)
        euler_tour.append(now)
dfs(0)

In [ ]:
# visited_time[v]: vを訪れた時間のリスト
visited_time = [[] for _ in range(N)]
for t, v in enumerate(euler_tour):
    visited_time[v].append(t)

# start_time[v]: 頂点vを初めて訪れた時間
# end_time[v]: 頂点vを最後に訪れた時間 の作成
start_time = [0] * N
end_time = [0] * N
for v in range(N):
    start_time[v] = visited_time[v][0]
    end_time[v] = visited_time[v][-1]

In [ ]:
# depth_time[d]: depth = d　の頂点v の start_time[v]の配列
depth_time = [[] for _ in range(N)]
for v in range(N):
    d = depth[v]
    depth_time[d].append(start_time[v])
for d in range(N):
    # 番兵を入れて, ソートする
    depth_time[d] = [-1] + sorted(depth_time[d]) + [10**10]

In [ ]:
from bisect import bisect_left, bisect_right
# 各クエリに対して, 頂点Uのs = start_time[U], e = end_time[U] とすると
# depth = D の 中で, [s, e]区間にstart_timeが含まれる頂点の個数が答え
# time_depth[D]に対して, s, eの挿入場所を binary searchで計算
for _ in range(Q):
    u, d = map(int, input().split())
    u -= 1
    s = start_time[u]
    e = end_time[u]
    left_ind = bisect_left(depth_time[d], e)
    right_ind = bisect_right(depth_time[d], e)
    print(right_ind - left_ind)

1 2
3
7 2
1
4 1
0
5 5
0


### 解法2: オイラーツアー (再帰DFS) + クエリ先読み + BIT
Pypyで通った. 再帰上限に注意

In [ ]:
import sys
sys.setrecursionlimit(7 * 10**5)

In [ ]:
# BIT
class BinaryIndexedTree:
    """
    A = [a0, a1, a2, ..., an-1]
    元のAの配列は0-indexだが, BIT上では1-indexで扱う
    """
    def __init__(self, n=10**6):
        self.size = n + 1
        self.tree = [0] * (n + 1)
        self.depth = n.bit_length()
        
    def update(self, i, x):
        """
        ai += x を する
        i: 0-index
        """
        # 1-indexに直す
        pos = i + 1
        while pos <= self.size:
            self.tree[pos] += x
            # 真上の位置は, iにiのLSBを加えたモノ
            pos += pos & -pos

    def sum(self, i):
        """
        a[0] + a[1] + ... + a[i] を 求める
        i は 0-index
        """
        pos = i + 1
        s = 0
        while pos > 0:
            s += self.tree[pos]
            # 左上は i に iのLSBを引いたモノ
            pos -= pos & -pos
        return s

    def sum_range(self, i, j):
        """
        a[i] + a[i+1] + ... + a[j] を 求める
        i, j は 0-index
        """
        return self.sum(j) - self.sum(i - 1)
    
    def lower_bound(self, x):
        """
        a0 + a1 + ... + ai >= x となる最小のiを取得.
        各項は非負である必要がある
        iは0 - index
        """
        if x <= 0:
            return -1
        
        k = 1 << (self.size.bit_length() - 1)
        pos = 0
        s = 0
        while k > 0:
            # (pos + kが配列の長さを超えない) and 和がxを超えない
            if (pos + k < self.size) and self.tree[pos + k] + s < x:
                s += self.tree[pos + k]
                pos += k
            # 1つ下の段に行く
            k //= 2
        return pos

In [ ]:
N = int(input())
P = list(map(int, input().split()))
Q = int(input())

In [ ]:
graph = [[] for _ in range(N)]
for i, p in enumerate(P, start=1):
    graph[p-1].append(i)

In [ ]:
# オイラーツアーとdepthの計算
euler_tour = []
depth = [0] * N
def dfs(now):
    euler_tour.append(now)
    for to in graph[now]:
        depth[to] = depth[now] + 1
        dfs(to)
        euler_tour.append(now)
dfs(0)

In [ ]:
# visited_time[v]: vを訪れた時間のリスト
visited_time = [[] for _ in range(N)]
for t, v in enumerate(euler_tour):
    visited_time[v].append(t)

# start_time[v]: 頂点vを初めて訪れた時間
# end_time[v]: 頂点vを最後に訪れた時間 の作成
start_time = [0] * N
end_time = [0] * N
for v in range(N):
    start_time[v] = visited_time[v][0]
    end_time[v] = visited_time[v][-1]

In [ ]:
# クエリ先読みして, depth順に処理する
query = [[] for _ in range(N)]
for i in range(Q):
    u, d = map(int, input().split())
    query[d].append([i, u-1])
ans = [0] * Q

In [ ]:
# depthごとに, そのdepthの頂点を初めて訪れた時間を記録
from collections import deque
depth_query = [deque() for _ in range(N)]
for v in range(N):
    d = depth[v]
    depth_query[d].append(start_time[v])

In [ ]:
bit = BinaryIndexedTree()

In [ ]:
# クエリ処理:
# 1. [L, R]区間に存在する, Depth (d - 1)以下の頂点の個数 h を計算
# 2. depth d の頂点に対して, bit[start_time[v]]に +1 加算
# 3. [L, R]区間に存在する, Depth d以下の頂点の個数 g を計算
# 4. クエリi に対して, g - h が答え
for d in range(N):
    if not query[d]: continue
    for i, u in query[d]:
        s, e = start_time[u], end_time[u]
        ans[i] -= bit.sum_range(s, e)
    
    dq = depth_query[d]
    while dq:
        time = dq.popleft()
        bit.update(time, 1)
    
    for i, u in query[d]:
        s, e = start_time[u], end_time[u]
        ans[i] += bit.sum_range(s, e)

In [ ]:
print(*ans)

### 解法3: クエリ先よみ + DFS

In [ ]:
import sys
from bisect import bisect_left
sys.setrecursionlimit(10**7)

In [ ]:
N = int(input())
P = list(map(int, input().split()))
Q = int(input())
graph = [[] for _ in range(N)]
for i, p in enumerate(P, start=1):
    graph[p-1].append(i)

7
1 1 2 2 4 2
4


In [ ]:
query = [[] for _ in range(N)]
ans = [0] * Q
for i in range(Q):
    u, d = map(int, input().split())
    query[u-1].append([i, d])

1 2
7 2
4 1
5 5


In [ ]:
depth_cnt = [0] * N
depth = [0] * N
def dfs(now):
    # クエリ処理: nowの部分木に入る前のdepth_cntの記録
    for i, d in query[now]:
        ans[i] -= depth_cnt[d]

    # depth_cntの更新
    depth_cnt[depth[now]] += 1
    # nowの部分木に突入
    for to in graph[now]:
        depth[to] = depth[now] + 1
        dfs(to)

    # クエリ処理: nowの部分木に入った後のdepth_cntの記録
    for i, d in query[now]:
        ans[i] += depth_cnt[d]

In [ ]:
dfs(0)

In [ ]:
print(*ans)

0 1 0 0
